In [ ]:
import numpy as np
import PIL
import math
import glob
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import csv

import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
import torch
import torch.nn as nn
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, label_path, transform=None):
        x = []
        y = []

        with open(label_path, 'r') as infh:
            for i, line in enumerate(infh):
                if i == 0:
                    continue
                d = line.replace('\n', '').split(',')
                x.append("../input/petfinder-pawpularity-score/train/"+d[0]+".jpg")
                y.append(float(d[-1]))
                
                self.x = x    
                self.y = torch.from_numpy(np.array(y)).float().view(-1, 1)
                self.transform = transform
                
    def __len__(self):
        return len(self.x)


    def __getitem__(self, i):
        img = PIL.Image.open(self.x[i]).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img, self.y[i]

In [ ]:
train_data_dir = "../input/petfinder-pawpularity-score/train.csv"

#画像整形
transform = torchvision.transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
])

#バッチサイズ
batchsize = 256

#train、validationのDataset作成
trainset = MyDataset(train_data_dir, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batchsize, shuffle=True)

In [ ]:
model = torchvision.models.vgg16(pretrained=False)
model.load_state_dict(torch.load("../input/pytorch-model-zoo/vgg16-397923af.pth"))
model.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),　　  
            nn.Linear(4096, 128),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(128, 1),
        )

for i, param in enumerate(model.parameters()):
    if i < 21:
        param.requires_grad = False

device = torch.device('cuda')
model.to(device)
        
criterion = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),0.16)
num_epoch = 50

In [ ]:
train_loss = []

for epoch in range(num_epoch):
    # 学習
    model.train()
    running_train_loss = 0.0
    with torch.set_grad_enabled(True):
        for data in trainloader:
            inputs, labels = data[0], data[1]
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            #forward
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_train_loss += loss.item() * inputs.size(0)
            
            #backward
            loss.backward()
            optimizer.step()
    train_loss.append(running_train_loss / len(trainset)) 

    print('#epoch:{}\ttrain loss: {}\ttrain Rloss: {}'.format(epoch + 1,
                                                running_train_loss / len(trainset), 
                                                round(np.sqrt(running_train_loss / len(trainset)),3),))

In [ ]:
# 次の平均、分散を用いて正規化
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225])
# 画像を256x256にリサイズ　→　中央の224を切り取り
preprocess = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])

img_list = glob.glob("../input/petfinder-pawpularity-score/test/*")

inf_out_list = []
inf_out_list.append(["Id","Pawpularity"])

with torch.no_grad():
    for img_file in img_list:
        img = Image.open(img_file)
        # 画像の正規化
        img_tensor = preprocess(img).to(device)

        model.eval()

        y = model(img_tensor.unsqueeze(0))

        inf_out_list.append([img_file.split("/")[-1].split(".")[0],round(y.item(),2)])
        

In [ ]:
with open("./submission.csv",mode="w") as f:
    writer = csv.writer(f)
    for row in inf_out_list:
        writer.writerow(row)